<a href="https://www.kaggle.com/code/kouroshsajjadi/smart-buildings-assignment-1?scriptVersionId=143859417" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import glob
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

# Read Building Energy Dataset

In [ ]:
path = '../input/building-energy-dataset'

# Use glob to get a list of all CSV files in the specified directory
all_files = glob.glob(path + "/*.csv")

# Initialize an empty list to store DataFrames
li = []

# Loop through each CSV file in the list of file paths
for filename in all_files:
    # Read the CSV file into a DataFrame
    # - 'filename': The name of the CSV file to be read
    # - 'index_col="Time"': Set the "Time" column as the index of the DataFrame
    # - 'parse_dates=True': Automatically parse date-like columns as datetime objects
    # - 'header=0': Use the first row of the CSV file as column headers
    df = pd.read_csv(filename, index_col="Time", parse_dates=True, header=0)
    
    # Append the DataFrame to the list of DataFrames
    li.append(df)

# Concatenate the list of DataFrames vertically (row-wise) into a single DataFrame
# - 'pd.concat()': Combine multiple DataFrames into one, stacking them vertically (axis=0)
# - 'li': List of individual DataFrames to concatenate
# - 'axis=0': Concatenate vertically
# - 'ignore_index=False': Preserve the original indices of the DataFrames
building = pd.concat(li, axis=0, ignore_index=False)

# Optionally, sort the DataFrame by the index if needed
# - 'building.sort_index()': Sort the DataFrame based on the index (row labels)
# - 'inplace=True': Apply the sorting operation directly to the 'building' DataFrame
building.sort_index(inplace=True)

#convert string index into datetime frame by using to_datetime() function
building.index = pd.to_datetime(building.index,format = "%d/%m/%Y %H:%M")

# Visualize Data

In [ ]:
# Display information about the concatenated DataFrame
# - 'building.info()': Print details about the resulting DataFrame, including data types and memory usage
building.info()

building.head()

In [ ]:
# Missingo on the raw data.
msno.matrix(building)

In [ ]:
# Show info about the total missing data.
missing_data = building.isna().sum()

# To calculate the total number of missing values
total_missing = missing_data.sum()

# To calculate the percentage of missing values
percentage_missing = (total_missing / (building.shape[0] * building.shape[1])) * 100

print("Total Missing Values:", total_missing)
print("Percentage of Missing Values:", percentage_missing, "%")

In [ ]:
# Report on rows having maximum and minimum missing values.

max_missing_index = building.isna().sum(axis=1).idxmax()
min_missing_index = building.isna().sum(axis=1).idxmin()

print("Index with Maximum Missing Values:", max_missing_index)
print("Index with Minimum Missing Values:", min_missing_index)

max_missing_row = building.loc[max_missing_index]
min_missing_row = building.loc[min_missing_index]

print("\nRow with maximum missing values.")
print(max_missing_row)

print("\nRow with minimum missing values.")
print(min_missing_row)

# pd.set_option('display.float_format', '{:.5f}'.format)
# row_with_values = building[building.notna().all(axis=1)].iloc[2000]
# print(row_with_values)

In [ ]:
msno.bar(building)

In [ ]:
# Show the missing data on each column.

# Assuming 'building' is your DataFrame
missing_data = building.isna().sum()

# Filter out columns with zero missing values
missing_data = missing_data[missing_data > 0]

# Calculate the total number of missing values
total_missing = missing_data.sum()

# Calculate the percentage of missing values
percentage_missing = (total_missing / (building.shape[0] * building.shape[1])) * 100

# Create a bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(missing_data.index, missing_data)
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.title('Number of Missing Values by Column')
plt.xticks(rotation=90)

# Add percentages on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval}\n{yval/total_missing*100:.2f}%', ha='center', va='bottom', color='black', fontsize=8)

# Add legend for total missing and percentage
legend_text = f'Total Missing: {total_missing}\nPercentage: {percentage_missing:.3f}%'
plt.legend([legend_text])

plt.show()


In [ ]:
# Report on columns having maximum and minimum missing values.

max_missing_index = building.isna().sum(axis=0).idxmax()
min_missing_index = building.isna().sum(axis=0).idxmin()

print("Index with Maximum Missing Values:", max_missing_index)
print("Index with Minimum Missing Values:", min_missing_index)

In [ ]:
# Plot scatter of the missing data
# Create a scatter plot for missing data
plt.figure(figsize=(20, 7))  # Set the figure size (optional)

# Iterate through each column
for col in building.columns:
    missing_data = building[col].isnull()
    if missing_data.any():  # Check if column has any missing values
        plt.scatter(building.index[missing_data], [col] * missing_data.sum(), marker='x', label=col)

size = 20
plt.xlabel('Date', fontsize=size)
plt.yticks(range(len(building.columns)), building.columns, fontsize=size)
plt.xticks(fontsize=size)
plt.title('Missing Data Scatter Plot', fontsize=size)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
msno.heatmap(building)

In [ ]:
msno.dendrogram(building)

In [ ]:
# Select rows where either 'feature1' or 'feature2' contains NaN values, 'HV light Power [kW], Power[kW]'
nan_rows = building[building[['Chiller Power [kW]', 'HVAC Actual [kW]', 'HV light Power [kW]', 'Power[kW]']].isna().all(axis=1)]

# Get the Datetime values for these rows
nan_dates = nan_rows.index
print(len(nan_dates))

# Print the Datetime values with missing values
print("Datetime values with missing values in either feature1 or feature2:")
print(nan_dates)

In [ ]:
building_linearly_interpolated = building.interpolate(method='linear')

row_before_interpolation = building.loc['2016-09-01 15:15:00']
row_after_interpolation = building_linearly_interpolated.loc['2016-09-01 15:15:00']

# Create a DataFrame to display the comparison
comparison_df = pd.DataFrame({'Before Interpolation': row_before_interpolation, 
                              'After Interpolation': row_after_interpolation})

# Display the comparison table
print(comparison_df)

row_before_interpolation = building.loc['2018-06-05 11:58:00']
row_after_interpolation = building_linearly_interpolated.loc['2018-06-05 11:58:00']

# Create a DataFrame to display the comparison
comparison_df = pd.DataFrame({'Before Interpolation': row_before_interpolation, 
                              'After Interpolation': row_after_interpolation})

# Display the comparison table
print(comparison_df)

In [ ]:
msno.matrix(building_linearly_interpolated)

# Show info about the total missing data on linearly interpolated data.
missing_data = building_linearly_interpolated.isna().sum()

# To calculate the total number of missing values
total_missing = missing_data.sum()

# To calculate the percentage of missing values
percentage_missing = (total_missing / (building_linearly_interpolated.shape[0] * building_linearly_interpolated.shape[1])) * 100

print("Total Missing Values:", total_missing)
print("Percentage of Missing Values:", percentage_missing, "%")

In [ ]:
# Weekly line plot.

columns_to_plot = ['HVAC Actual [kW]', 'Chiller Power [kW]', 'Humidifier power [kW]', 'HV light Power [kW]', 'Power[kW]']

# Calculate the weekly rolling averages for each column
weekly_avg = building_linearly_interpolated[columns_to_plot].resample('W').mean()

# Calculate the sum of the power consumption values for each week
weekly_sum = weekly_avg.sum(axis=1)

# Plot the weekly rolling averages
plt.figure(figsize=(12, 6))

for col in columns_to_plot:
    plt.plot(weekly_avg.index, weekly_avg[col], label=col)

# Add a line for the sum
plt.plot(weekly_avg.index, weekly_sum, label='Total Power Consumption', color='black', linestyle='dashed')

plt.xlabel('Date')
plt.ylabel('Power Consumption [kW]')
plt.title('Weekly Rolling Average Power Consumption')
plt.legend()
plt.show()

In [ ]:
# Weekly bar chart.

# Define the time period (e.g., a specific week)
start_date = '2016-09-01'
end_date = '2016-09-07'

# Filter the data for the specified time period
period_data = building_linearly_interpolated.loc[start_date:end_date]

# Define the columns to plot
columns_to_plot = ['HVAC Actual [kW]', 'Chiller Power [kW]', 'Humidifier power [kW]', 'HV light Power [kW]', 'Power[kW]']

# Plot the bar chart
plt.figure(figsize=(12, 6))
plt.bar(columns_to_plot, period_data[columns_to_plot].mean(), color='blue')
plt.xlabel('Features')
plt.ylabel('Average Power Consumption [kW]')
plt.title(f'Average Power Consumption for Week of {start_date} to {end_date}')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Weekly box plot.

# Define the columns to plot
columns_to_plot = ['HVAC Actual [kW]', 'Chiller Power [kW]', 'Humidifier power [kW]', 'HV light Power [kW]', 'Power[kW]']

# Create a box plot
plt.figure(figsize=(10, 6))
sns.boxplot(data=building_linearly_interpolated[columns_to_plot])
plt.xlabel('Features')
plt.ylabel('Power Consumption [kW]')
plt.title('Distribution of Power Consumption for Each Feature')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Define the columns representing energy consumption
energy_columns = ['HVAC Actual [kW]', 'Chiller Power [kW]', 'Humidifier power [kW]', 'HV light Power [kW]', 'Power[kW]']

# Resample data by week and calculate the range
weekly_range = building_linearly_interpolated[energy_columns].resample('W').apply(lambda x: x.max() - x.min())

# Print the range of energy consumption per week
print(weekly_range)

In [ ]:
# Calculate the yearly consumption for each component
yearly_hvac = building_linearly_interpolated['HVAC Actual [kW]'].resample('Y').sum()
yearly_chiller = building_linearly_interpolated['Chiller Power [kW]'].resample('Y').sum()
yearly_humidifier = building_linearly_interpolated['Humidifier power [kW]'].resample('Y').sum()
yearly_hv_light = building_linearly_interpolated['HV light Power [kW]'].resample('Y').sum()

# Calculate the total Power [kW] for each year
yearly_total_power = building_linearly_interpolated['Power[kW]'].resample('Y').sum()

# Compare the total Power [kW] with the sum of components
for year in yearly_hvac.index:
    total_power_year = yearly_total_power.loc[year]
    components_sum = (
        yearly_hvac.loc[year] +
        yearly_chiller.loc[year] +
        yearly_humidifier.loc[year] +
        yearly_hv_light.loc[year]
    )
    print(f'Year {year.year}:')
    print(f'Total Power [kW]: {total_power_year:.2f}')
    print(f'Sum of Components: {components_sum:.2f}')
    print(f'Difference: {total_power_year - components_sum:.2f}\n')

In [ ]:
#resample the data by daily
building_day = building_linearly_interpolated.resample('D').mean()

In [ ]:
#select year
select_year = 2018
start_time = pd.to_datetime(f'{select_year}-1-1')
end_time = pd.to_datetime(f'{select_year+1}-1-1')


building_year_select = building_day.loc[start_time:end_time]
#building_year_select.head(5)

#extract data of AHU    'HVAC Actual [kW]','Chiller Power [kW]','HV light Power [kW]' ,'Humidifier power [kW]', 
AHU_level_data= building_year_select[['HVAC Actual [kW]','Humidifier power [kW]','Chiller Power [kW]']]
#AHU_level_data.head()
#Plot the energy consumption of AHU in a line chart
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed

fontsize = 15

for col in AHU_level_data.columns:
    plt.plot(AHU_level_data.index, AHU_level_data[col], label= col)

plt.title(f'Daily Energy consumption of AHU at component-level performance in {select_year} ',fontsize = fontsize)
plt.xlabel('Time', fontsize = fontsize)
plt.ylabel('kW', fontsize = fontsize)
plt.xticks(fontsize = fontsize)
plt.yticks(fontsize = fontsize)
#plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend(fontsize = fontsize)

plt.tight_layout()
plt.grid()

plt.show()

In [ ]:
# Loop over the selected years (from 2016 to 2021)
for select_year in range(2016, 2022):
    start_time = pd.to_datetime(f'{select_year}-1-1')
    end_time = pd.to_datetime(f'{select_year+1}-1-1')

    building_year_select = building_day.loc[start_time:end_time]

    # Extract data of AHU
    AHU_level_data = building_year_select[['HVAC Actual [kW]', 'Humidifier power [kW]', 'Chiller Power [kW]']]

    # Plot the energy consumption of AHU in a line chart
    plt.figure(figsize=(10, 6))

    for col in AHU_level_data.columns:
        plt.plot(AHU_level_data.index, AHU_level_data[col], label=col)

    plt.title(f'Daily Energy consumption of AHU at component-level performance in {select_year}', fontsize=fontsize)
    plt.xlabel('Time', fontsize=fontsize)
    plt.ylabel('kW', fontsize=fontsize)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    plt.legend(fontsize=fontsize)

    plt.tight_layout()
    plt.grid()

    plt.show()
